In [22]:
!rm -r *

rm: cannot remove '*': No such file or directory


In [23]:
# Run this cell and select the kaggle.json file downloaded
# from the Kaggle account settings page.
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"kingomalek","key":"aa7c8c72a5e7fab0b7e9139cf4e6b0c9"}'}

In [24]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 66 Aug 30 09:10 kaggle.json


In [0]:
# Next, install the Kaggle API client.
!pip install -q kaggle

In [0]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [27]:
!kaggle datasets download -d alexattia/the-simpsons-characters-dataset

 99%|██████████████████████████████████████▌| 1.05G/1.07G [00:07<00:00, 167MB/s]
100%|███████████████████████████████████████| 1.07G/1.07G [00:07<00:00, 151MB/s]


In [0]:
!unzip -q the-simpsons-characters-dataset.zip -d dataset


In [0]:
!rm  the-simpsons-characters-dataset.zip

In [0]:
!unzip -q dataset/simpsons_dataset.zip -d dataset2


In [0]:
!rm -r dataset

In [34]:
import numpy as np
import pandas as pd
import keras
from keras.models import Sequential #model of the NN
from keras.layers import Dense, Dropout, Activation, Flatten, Reshape #core layers
from keras.layers import Convolution2D, MaxPooling2D #convolutionnal layers
from keras.datasets import fashion_mnist
from keras.utils import np_utils



Using TensorFlow backend.


In [45]:
model = Sequential()
model.add(Convolution2D(25, 3, 3, activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2,2)))  
model.add(Dropout(0.1))
model.add(Convolution2D(50, 3, 3, activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D((2,2)))  
model.add(Dropout(0.1))
model.add(Flatten())
model.add(Dense(output_dim=100))
model.add(Dropout(0.1))
model.add(Activation('relu'))
model.add(Dense(output_dim=42))
model.add(Activation('softmax'))

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(25, (3, 3), activation="relu", input_shape=(128, 128,...)`
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(50, (3, 3), activation="relu", input_shape=(128, 128,...)`
  """
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:9: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=100)`
  if __name__ == '__main__':
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:12: UserWarning: Update your `Dense` call to the Keras 2 API: `Dense(units=42)`
  if sys.path[0] == '':


In [0]:
import os
import glob
import cv2
X = []
Y = []
dirs =  glob.glob("/dataset2/simpsons_dataset/*")
types = ('*.jpg', '*.jpeg','*.png') 
for d in dirs:
    print(d)
    label = d.split('/')[-1]
    images = []
    for typ in types:
        images += glob.glob(d+"/*"+typ)
    for i in range(len(images)):
        #print(images[i])
        image = cv2.imread(images[i])
        image = cv2.cvtColor(image,cv2.COLOR_BGR2RGB)
        res = cv2.resize(image,(224, 224), interpolation = cv2.INTER_CUBIC)
        X.append(res)
        Y.append(label)
        

/dataset2/simpsons_dataset/comic_book_guy
/dataset2/simpsons_dataset/jasper_beardly
/dataset2/simpsons_dataset/sideshow_mel
/dataset2/simpsons_dataset/otto_mann
/dataset2/simpsons_dataset/agnes_skinner
/dataset2/simpsons_dataset/hans_moleman
/dataset2/simpsons_dataset/troy_mcclure
/dataset2/simpsons_dataset/marge_simpson
/dataset2/simpsons_dataset/ralph_wiggum
/dataset2/simpsons_dataset/selma_bouvier
/dataset2/simpsons_dataset/helen_lovejoy
/dataset2/simpsons_dataset/snake_jailbird
/dataset2/simpsons_dataset/ned_flanders
/dataset2/simpsons_dataset/principal_skinner


In [0]:
X = np.asarray(X)
Y = np.asarray(Y)

In [0]:
from sklearn.utils import shuffle
X, Y = shuffle(X, Y, random_state=0)

In [41]:
X.shape

(20933, 128, 128, 3)

In [0]:
np.save("X", X)
np.save("Y", Y)

In [0]:
X.shape

(1115, 128, 128, 3)

In [0]:
Y.shape

(1115,)

In [0]:
X = X.reshape((-1, 224, 224, 3))
X = X.astype('float32')
X = X/255

In [0]:
mp = dict(zip(list(set(Y)),range(0,len(list(set(Y))))))
Y_ = []
for y in Y:
    Y_.append(mp[y])
Y  = np_utils.to_categorical(np.asarray(Y_))


In [46]:
#prepare the model
model.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy']
)

print(model.summary())

#train the model
history = model.fit(X, Y, batch_size=64, epochs=20,shuffle=True,validation_split=0.2)

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 126, 126, 25)      700       
_________________________________________________________________
max_pooling2d_3 (MaxPooling2 (None, 63, 63, 25)        0         
_________________________________________________________________
dropout_4 (Dropout)          (None, 63, 63, 25)        0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 61, 61, 50)        11300     
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 30, 30, 50)        0         
_________________________________________________________________
dropout_5 (Dropout)          (None, 30, 30, 50)        0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 45000)             0         
__________

In [53]:
VGG16 = keras.applications.vgg16.VGG16(include_top=True,input_tensor=None, input_shape=(224,224,3), pooling=None, classes=42)
#prepare the model
VGG16.compile(
    optimizer='adam',
    loss='categorical_crossentropy',
    metrics=['accuracy'])
histo_VGG16 = VGG16.fit(X, Y, batch_size=64, epochs=20,shuffle=True,validation_split=0.2)

80142336/80134624 [==============================] - 2s 0us/step


ValueError: ignored